<a href="https://colab.research.google.com/github/SrijaG29/Movie-Data-Analysis-and-Recommendations-Using-PySpark/blob/main/Movies_ratings_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To create a project using the movie and ratings datasets with PySpark, here’s a step-by-step guide along with some ideas for questions you can answer. This will help demonstrate your PySpark skills and data analysis capabilities on your resume.

### Step 1: Data Exploration
1. **Load the datasets into PySpark**:
   - Load the movie dataset (`movies.csv`) and ratings dataset (`ratings.csv`) into PySpark DataFrames.

2. **Inspect the Data**:
   - Display the schema and first few rows of both DataFrames to understand the data structure.

### Step 2: Data Cleaning and Preparation
1. **Handle Missing Values**:
   - Check for any missing values and decide how to handle them (e.g., drop rows, fill with default values).

2. **Data Transformation**:
   - Extract the year from the movie title and create a new column `year`.
   - Split the `genres` column into an array of genres.

3. **Join the Datasets**:
   - Perform an inner join on the `movieId` column to combine the movie and ratings datasets.

### Step 3: Data Analysis
1. **Top-Rated Movies**:
   - Find the top 10 movies with the highest average rating.

2. **Popular Genres**:
   - Identify the most popular genres based on the number of ratings.

3. **User Behavior**:
   - Analyze the distribution of ratings by users. For example, find how many movies each user has rated.

4. **Yearly Trends**:
   - Determine how the average movie rating has changed over the years.

5. **Genre-Based Recommendations**:
   - For a given genre, list the top 5 movies based on average ratings.

6. **Movies with the Most Reviews**:
   - Identify the movies with the highest number of ratings.

### Step 4: Advanced Analysis
1. **User-Specific Recommendations**:
   - Build a basic recommendation system by suggesting top-rated movies that a user has not rated yet.

2. **Correlate Ratings and Release Year**:
   - Analyze if there’s any correlation between the release year and the average rating of movies.

3. **Genre Diversity in Top-Rated Movies**:
   - Examine the genre diversity among the top 100 highest-rated movies.

### Step 5: Performance Optimization
1. **Cache and Persist**:
   - Use caching and persistence in PySpark to optimize the performance of your queries.

2. **Partitioning**:
   - Apply partitioning to the data to improve the efficiency of operations, especially for large datasets.

### Step 6: Visualization
1. **Visualize Data**:
   - Use PySpark with an external library like Matplotlib, Seaborn, or even Power BI to create visualizations such as:
     - Distribution of ratings.
     - Average rating per genre.
     - Trends in movie ratings over the years.

### Conclusion
This project structure not only demonstrates your technical ability to handle and analyze data using PySpark but also your ability to derive meaningful insights from complex datasets. Once complete, you can showcase this project in your portfolio or resume, highlighting key aspects such as data cleaning, transformation, analysis, and visualization.

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=ca950002024c4734293f36fafb782435d4b8a7d004c953813d6511b8247e3d15
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = (
    SparkSession
    .builder
    .appName("Movie ratings")
    .master("local[*]")
    .getOrCreate()
)

In [4]:
spark

**Step 1: Data Exploration**
Load the datasets into PySpark session **"spark"**:

Load the movie dataset (movies.csv) and ratings dataset (ratings.csv) into PySpark DataFrames.
Inspect the Data:

Display the schema and first few rows of both DataFrames to understand the data structure.

In [5]:
movies_df = spark.read.format("csv").option("header",True).load("/content/movies.csv")
movies_df.show(truncate = False)

+-------+-------------------------------------+-------------------------------------------+
|movieId|title                                |genres                                     |
+-------+-------------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                       |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)              |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)             |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)   |Comedy                                     |
|6      |Heat (1995)                          |Action|Crime|Thriller                      |
|7      |Sabrina (1995)                       |Comedy|Romance                             |
|8      |Tom and Huck (1995)                  |Adventure|Children               

In [6]:
ratings_df = spark.read.format("csv").option("header",True).load("/content/ratings.csv")
ratings_df.show(truncate = False)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|1     |17     |4.0   |944249077|
|1     |25     |1.0   |944250228|
|1     |29     |2.0   |943230976|
|1     |30     |5.0   |944249077|
|1     |32     |5.0   |943228858|
|1     |34     |2.0   |943228491|
|1     |36     |1.0   |944249008|
|1     |80     |5.0   |944248943|
|1     |110    |3.0   |943231119|
|1     |111    |5.0   |944249008|
|1     |161    |1.0   |943231162|
|1     |166    |5.0   |943228442|
|1     |176    |4.0   |944079496|
|1     |223    |3.0   |944082810|
|1     |232    |5.0   |943228442|
|1     |260    |5.0   |943228696|
|1     |302    |4.0   |944253272|
|1     |306    |5.0   |944248888|
|1     |307    |5.0   |944253207|
|1     |322    |4.0   |944053801|
+------+-------+------+---------+
only showing top 20 rows



**Step 2: Data Cleaning and Preparation**

**Handle Missing Values:**

Check for any missing values and decide how to handle them (e.g., drop rows, fill with default values).

In [7]:
from pyspark.sql.functions import sum,col,when,split

In [8]:
print(movies_df.dtypes)

[('movieId', 'string'), ('title', 'string'), ('genres', 'string')]


In [9]:
print(ratings_df.dtypes)

[('userId', 'string'), ('movieId', 'string'), ('rating', 'string'), ('timestamp', 'string')]


In [10]:
# Create an empty list to hold column expressions for missing values
missing_value_expressions_movies = []

# Loop through each column in the DataFrame
for column in movies_df.columns:
    # Count the number of null values in the current column
    missing_expr = sum(when(col(column).isNull(), 1).otherwise(0)).alias(column)
    missing_value_expressions_movies.append(missing_expr)

# Aggregate the missing value counts
missing_movies_df = movies_df.agg(*missing_value_expressions_movies)

# Show the results
missing_movies_df.show()

+-------+-----+------+
|movieId|title|genres|
+-------+-----+------+
|      0|    0|     0|
+-------+-----+------+



In [11]:
# Create an empty list to hold column expressions for missing values
missing_value_expressions_ratings = []

# Loop through each column in the DataFrame
for column in ratings_df.columns:
    # Count the number of null values in the current column
    missing_expr = sum(when(col(column).isNull(), 1).otherwise(0)).alias(column)
    missing_value_expressions_ratings.append(missing_expr)

# Aggregate the missing value counts
missing_ratings_df = ratings_df.agg(*missing_value_expressions_ratings)

# Show the results
missing_ratings_df.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     0|      0|     0|        0|
+------+-------+------+---------+



**" There are no null values in these datasets. "**

**Data Transformation:**

Extract the year from the movie title and create a new column year. Split the genres column into an array of genres.

In [12]:
movies_df.show(truncate = False)

+-------+-------------------------------------+-------------------------------------------+
|movieId|title                                |genres                                     |
+-------+-------------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                       |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)              |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)             |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)   |Comedy                                     |
|6      |Heat (1995)                          |Action|Crime|Thriller                      |
|7      |Sabrina (1995)                       |Comedy|Romance                             |
|8      |Tom and Huck (1995)                  |Adventure|Children               

Extracting year from title.

In [13]:
from pyspark.sql.functions import expr,trim

In [14]:
movies_df = movies_df.withColumn("movie_name",split(col("title"),"\\(").getItem(0))\
                     .withColumn("year", expr("substring_index(title, '(', -1)"))\
                     .withColumn("year", trim(col("year")).substr(1, 4))

movies_df.show(truncate=False)

+-------+-------------------------------------+-------------------------------------------+-------------------------------+----+
|movieId|title                                |genres                                     |movie_name                     |year|
+-------+-------------------------------------+-------------------------------------------+-------------------------------+----+
|1      |Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|Toy Story                      |1995|
|2      |Jumanji (1995)                       |Adventure|Children|Fantasy                 |Jumanji                        |1995|
|3      |Grumpier Old Men (1995)              |Comedy|Romance                             |Grumpier Old Men               |1995|
|4      |Waiting to Exhale (1995)             |Comedy|Drama|Romance                       |Waiting to Exhale              |1995|
|5      |Father of the Bride Part II (1995)   |Comedy                                     |Father

Here for some movies there is no year mentioned so I am deleting those rows.

In [15]:
movies_df = movies_df.filter(col("year").rlike("^[0-9]{4}$"))
movies_df.show()

+-------+--------------------+--------------------+--------------------+----+
|movieId|               title|              genres|          movie_name|year|
+-------+--------------------+--------------------+--------------------+----+
|      1|    Toy Story (1995)|Adventure|Animati...|          Toy Story |1995|
|      2|      Jumanji (1995)|Adventure|Childre...|            Jumanji |1995|
|      3|Grumpier Old Men ...|      Comedy|Romance|   Grumpier Old Men |1995|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|  Waiting to Exhale |1995|
|      5|Father of the Bri...|              Comedy|Father of the Bri...|1995|
|      6|         Heat (1995)|Action|Crime|Thri...|               Heat |1995|
|      7|      Sabrina (1995)|      Comedy|Romance|            Sabrina |1995|
|      8| Tom and Huck (1995)|  Adventure|Children|       Tom and Huck |1995|
|      9| Sudden Death (1995)|              Action|       Sudden Death |1995|
|     10|    GoldenEye (1995)|Action|Adventure|...|          Gol

In [16]:
# dropping the column title

movies_df = movies_df.drop(movies_df.title)
movies_df.show(truncate = False)

+-------+-------------------------------------------+-------------------------------+----+
|movieId|genres                                     |movie_name                     |year|
+-------+-------------------------------------------+-------------------------------+----+
|1      |Adventure|Animation|Children|Comedy|Fantasy|Toy Story                      |1995|
|2      |Adventure|Children|Fantasy                 |Jumanji                        |1995|
|3      |Comedy|Romance                             |Grumpier Old Men               |1995|
|4      |Comedy|Drama|Romance                       |Waiting to Exhale              |1995|
|5      |Comedy                                     |Father of the Bride Part II    |1995|
|6      |Action|Crime|Thriller                      |Heat                           |1995|
|7      |Comedy|Romance                             |Sabrina                        |1995|
|8      |Adventure|Children                         |Tom and Huck                   |1995|

In [17]:
genres_df = movies_df.select(movies_df.movieId,movies_df.genres)
genres_df.show(truncate = False)

+-------+-------------------------------------------+
|movieId|genres                                     |
+-------+-------------------------------------------+
|1      |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Adventure|Children|Fantasy                 |
|3      |Comedy|Romance                             |
|4      |Comedy|Drama|Romance                       |
|5      |Comedy                                     |
|6      |Action|Crime|Thriller                      |
|7      |Comedy|Romance                             |
|8      |Adventure|Children                         |
|9      |Action                                     |
|10     |Action|Adventure|Thriller                  |
|11     |Comedy|Drama|Romance                       |
|12     |Comedy|Horror                              |
|13     |Adventure|Animation|Children               |
|14     |Drama                                      |
|15     |Action|Adventure|Romance                   |
|16     |Crime|Drama        

In [18]:
genres_df = genres_df.withColumn("gener",split(genres_df.genres,"\|"))
genres_df.show(truncate = False)

+-------+-------------------------------------------+-------------------------------------------------+
|movieId|genres                                     |gener                                            |
+-------+-------------------------------------------+-------------------------------------------------+
|1      |Adventure|Animation|Children|Comedy|Fantasy|[Adventure, Animation, Children, Comedy, Fantasy]|
|2      |Adventure|Children|Fantasy                 |[Adventure, Children, Fantasy]                   |
|3      |Comedy|Romance                             |[Comedy, Romance]                                |
|4      |Comedy|Drama|Romance                       |[Comedy, Drama, Romance]                         |
|5      |Comedy                                     |[Comedy]                                         |
|6      |Action|Crime|Thriller                      |[Action, Crime, Thriller]                        |
|7      |Comedy|Romance                             |[Comedy, Ro

In [19]:
genres_df = genres_df.drop(genres_df.genres)
genres_df.show(truncate = False)

+-------+-------------------------------------------------+
|movieId|gener                                            |
+-------+-------------------------------------------------+
|1      |[Adventure, Animation, Children, Comedy, Fantasy]|
|2      |[Adventure, Children, Fantasy]                   |
|3      |[Comedy, Romance]                                |
|4      |[Comedy, Drama, Romance]                         |
|5      |[Comedy]                                         |
|6      |[Action, Crime, Thriller]                        |
|7      |[Comedy, Romance]                                |
|8      |[Adventure, Children]                            |
|9      |[Action]                                         |
|10     |[Action, Adventure, Thriller]                    |
|11     |[Comedy, Drama, Romance]                         |
|12     |[Comedy, Horror]                                 |
|13     |[Adventure, Animation, Children]                 |
|14     |[Drama]                        

**Join the Datasets:**

Perform an inner join on the movieId column to combine the movie and ratings datasets.

In [20]:
movies_ratings_df = movies_df.join(ratings_df,on='movieId',how='inner')
movies_ratings_df.show(truncate = False)

+-------+--------------------------------------+-----------------------------------+----+------+------+---------+
|movieId|genres                                |movie_name                         |year|userId|rating|timestamp|
+-------+--------------------------------------+-----------------------------------+----+------+------+---------+
|17     |Drama|Romance                         |Sense and Sensibility              |1995|1     |4.0   |944249077|
|25     |Drama|Romance                         |Leaving Las Vegas                  |1995|1     |1.0   |944250228|
|29     |Adventure|Drama|Fantasy|Mystery|Sci-Fi|City of Lost Children, The         |1995|1     |2.0   |943230976|
|30     |Crime|Drama                           |Shanghai Triad                     |1995|1     |5.0   |944249077|
|32     |Mystery|Sci-Fi|Thriller               |Twelve Monkeys                     |1995|1     |5.0   |943228858|
|34     |Children|Drama                        |Babe                               |1995

**Step 3: Data Analysis**



**Top-Rated Movies:**

Find the top 10 movies with the highest average rating.

In [21]:
from pyspark.sql.functions import avg,desc,asc
top_ten_movies = movies_ratings_df.groupby('movieId','movie_name').agg(avg('rating').alias('Avg_rating'))
top_ten_movies = top_ten_movies.orderBy(top_ten_movies.Avg_rating.desc())
top_ten_movies.show(10,truncate = False)

+-------+-----------------------------------------------+----------+
|movieId|movie_name                                     |Avg_rating|
+-------+-----------------------------------------------+----------+
|148084 |Emmanuelle in Soho                             |5.0       |
|166267 |Finnish Blood Swedish Heart                    |5.0       |
|268214 |Where Is Anne Frank                            |5.0       |
|136878 |Paradh                                         |5.0       |
|126959 |The Epic of Everest                            |5.0       |
|152916 |Scene from the Elevator Ascending Eiffel Tower |5.0       |
|287247 |The Beach Boys: Making Pet Sounds              |5.0       |
|256477 |Justice League: Secret Origins                 |5.0       |
|203882 |Dead in the Water                              |5.0       |
|208090 |The Fallen of World War II                     |5.0       |
+-------+-----------------------------------------------+----------+
only showing top 10 rows



**Popular Genres:**

Identify the most popular genres based on the number of ratings.


In [22]:
from pyspark.sql.functions import count

popular_genres = movies_ratings_df.groupBy('genres').agg(count('rating').alias('No_of_ratings'))
popular_genres = popular_genres.sort(popular_genres.No_of_ratings.desc())
popular_genres.show(truncate = False)

+--------------------------------+-------------+
|genres                          |No_of_ratings|
+--------------------------------+-------------+
|Drama                           |464344       |
|Comedy                          |389335       |
|Comedy|Romance                  |225989       |
|Drama|Romance                   |202074       |
|Comedy|Drama                    |191448       |
|Comedy|Drama|Romance            |180967       |
|Action|Adventure|Sci-Fi         |164646       |
|Crime|Drama                     |157048       |
|Action|Crime|Thriller           |99870        |
|Drama|Thriller                  |96289        |
|Action|Adventure|Sci-Fi|Thriller|90295        |
|Action|Adventure|Thriller       |85477        |
|Action|Sci-Fi|Thriller          |81405        |
|Crime|Drama|Thriller            |80534        |
|Drama|War                       |73344        |
|Action|Crime|Drama|Thriller     |67709        |
|Comedy|Crime                    |66584        |
|Action|Drama|War   

**User Behavior:**

Analyze the distribution of ratings by users. For example, find how many movies each user has rated.


In [23]:
user_rating = movies_ratings_df.groupBy('userId').agg(count('userId').alias('No_of_ratings')).orderBy(movies_ratings_df.userId.asc())
user_rating.show(truncate = False)

+------+-------------+
|userId|No_of_ratings|
+------+-------------+
|1     |141          |
|10    |660          |
|100   |248          |
|1000  |27           |
|10000 |165          |
|10001 |25           |
|10002 |35           |
|10003 |447          |
|10004 |212          |
|10005 |420          |
|10006 |63           |
|10007 |89           |
|10008 |91           |
|10009 |59           |
|1001  |102          |
|10010 |137          |
|10011 |20           |
|10012 |64           |
|10013 |30           |
|10014 |88           |
+------+-------------+
only showing top 20 rows



**Yearly Trends:**

Determine how the average movie rating has changed over the years.


In [24]:
movies_ratings_df.select('year').show()

+----+
|year|
+----+
|1995|
|1995|
|1995|
|1995|
|1995|
|1995|
|1995|
|1995|
|1995|
|1976|
|1995|
|1995|
|1995|
|1994|
|1994|
|1977|
|1994|
|1994|
|1993|
|1995|
+----+
only showing top 20 rows



In [25]:
avg_rating_year = movies_ratings_df.groupBy('year').agg(avg('rating').alias('Avg_rating'))
avg_rating_year.show(truncate = False)

+----+------------------+
|year|Avg_rating        |
+----+------------------+
|1953|3.704188226105633 |
|1903|3.015625          |
|1957|4.027803752883237 |
|1897|2.7714285714285714|
|1987|3.552403055165125 |
|1956|3.720171773444754 |
|2016|3.5458345292766933|
|1936|3.8535169210351694|
|2012|3.541644356393232 |
|2020|3.3205756276791183|
|1958|3.818285762683154 |
|1910|2.789473684210526 |
|1943|3.6750700280112043|
|1915|2.9939024390243905|
|1972|3.9712906024823633|
|1931|3.9088599878073564|
|1988|3.531758569944598 |
|1938|3.8876037213980386|
|1926|3.851029252437703 |
|1911|3.566666666666667 |
+----+------------------+
only showing top 20 rows



**Genre-Based Recommendations:**

For a given genre, list the top 5 movies based on average ratings.


In [26]:
genre_input = input("Enter the genre to filter movies: ")

top_five_movies = (movies_ratings_df.filter(col('genres') == genre_input)\
                   .groupBy('movie_name')\
                   .agg(avg('rating').alias('Avg_rating'))\
                   .orderBy(col('Avg_rating').desc())
                   .limit(5))
top_five_movies.show(truncate = False)


Enter the genre to filter movies: Action
+----------------------------+----------+
|movie_name                  |Avg_rating|
+----------------------------+----------+
|Dreadnaught                 |5.0       |
|Drunken Master 3            |5.0       |
|The Division: Agent Origins |5.0       |
|Buffalo Girls               |5.0       |
|Reich                       |5.0       |
+----------------------------+----------+



In [27]:
x = movies_df.filter(col("title").like("%Dreadnaught%"))
x.select(x.genres).show()

+------+
|genres|
+------+
|Action|
+------+



**Movies with the Most Reviews:**

Identify the movies with the highest number of ratings.

In [28]:
from pyspark.sql.functions import desc

In [29]:
most_reviews = movies_ratings_df.groupBy('movieId','movie_name')\
              .agg(count(col('rating')).alias('No_of_reviews'))\
              .orderBy(col('No_of_reviews').desc())\
              .limit(1)
most_reviews.show(truncate = False)

+-------+--------------------------+-------------+
|movieId|movie_name                |No_of_reviews|
+-------+--------------------------+-------------+
|318    |Shawshank Redemption, The |21119        |
+-------+--------------------------+-------------+



In [30]:
x = movies_df.filter(col("title").like("Shawshank Redemption%"))
x.show(truncate = False)

+-------+-----------+--------------------------+----+
|movieId|genres     |movie_name                |year|
+-------+-----------+--------------------------+----+
|318    |Crime|Drama|Shawshank Redemption, The |1994|
+-------+-----------+--------------------------+----+



**Step 4: Advanced Analysis**


**User-Specific Recommendations:**

Build a basic recommendation system by suggesting top-rated movies that a user has not rated yet.


In [44]:
top_rated_movies = movies_ratings_df.groupBy('movieId', 'movie_name')\
    .agg(avg(col('rating')).alias('Avg_rating'))\
    .orderBy(col('Avg_rating').desc())\
    .limit(100)\
    .select('movieId','movie_name')

top_rated_movies.show(truncate = False)

+-------+--------------------------------------------------------------------------------+
|movieId|movie_name                                                                      |
+-------+--------------------------------------------------------------------------------+
|208090 |The Fallen of World War II                                                      |
|268214 |Where Is Anne Frank                                                             |
|82467  |Forever Amber                                                                   |
|278754 |One Piece: Episode of Chopper Plus: Bloom in the Winter, Miracle Cherry Blossom |
|168558 |Finding Rin Tin Tin                                                             |
|287247 |The Beach Boys: Making Pet Sounds                                               |
|267928 |Unreal News Reel                                                                |
|120867 |Tropico                                                                         |

In [35]:
total_users = movies_ratings_df.select('userId').distinct().count()
print(total_users)

41195


In [45]:
user_id_input = input('Enter user id')

user_ratings = movies_ratings_df.filter(col('userId') == user_id_input).select('movieId','movie_name')
user_ratings.show(truncate = False)

Enter user id1
+-------+-----------------------------------+
|movieId|movie_name                         |
+-------+-----------------------------------+
|17     |Sense and Sensibility              |
|25     |Leaving Las Vegas                  |
|29     |City of Lost Children, The         |
|30     |Shanghai Triad                     |
|32     |Twelve Monkeys                     |
|34     |Babe                               |
|36     |Dead Man Walking                   |
|80     |White Balloon, The                 |
|110    |Braveheart                         |
|111    |Taxi Driver                        |
|161    |Crimson Tide                       |
|166    |Doom Generation, The               |
|176    |Living in Oblivion                 |
|223    |Clerks                             |
|232    |Eat Drink Man Woman                |
|260    |Star Wars: Episode IV - A New Hope |
|302    |Queen Margot                       |
|306    |Three Colors: Red                  |
|307    |Three Colo


The **left anti join** in PySpark is similar to the join functionality, but it returns only columns from the left DataFrame for non-matched records.

In [46]:
recommended_movies_df = top_rated_movies.join(user_ratings, on='movieId', how='left_anti')
recommended_movies_df.show(truncate = False)

+-------+----------------------------------------------------------------------+
|movieId|movie_name                                                            |
+-------+----------------------------------------------------------------------+
|250648 |From Hare to Heir                                                     |
|280162 |Unrest                                                                |
|216372 |Adoring                                                               |
|199710 |L'arroseur arrosé                                                     |
|161818 |VeggieTales: Duke and the Great Pie War                               |
|134037 |War-Time Romance                                                      |
|83558  |On the Occasion of Remembering the Turning Gate                       |
|267940 |Silvery Moon                                                          |
|251616 |Breaking Boundaries: The Science of Our Planet                        |
|126959 |The Epic of Everest

left_anti join will keep only the movies that are not in the user’s rated movies list.

**Correlate Ratings and Release Year:**

Analyze if there’s any correlation between the release year and the average rating of movies.


**Genre Diversity in Top-Rated Movies:**

Examine the genre diversity among the top 100 highest-rated movies.

**Step 5: Performance Optimization**


**Cache and Persist:**

Use caching and persistence in PySpark to optimize the performance of your queries.


**Partitioning:**

Apply partitioning to the data to improve the efficiency of operations, especially for large datasets.


**Step 6: Visualization**

**Visualize Data:**
Use PySpark with an external library like Matplotlib, Seaborn, or even Power BI to create visualizations such as:
Distribution of ratings.
Average rating per genre.
Trends in movie ratings over the years.